In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("LAB6")
sc = SparkContext(conf=conf)

In [2]:
inputpath = "ReviewsSample.csv"
outputpath1 = "LAB6_2024_res_1"
outputpath2 = "LAB6_2024_res_2"
outputpath3 = "LAB6_2024_res_3"

In [4]:
inputRDD = sc.textFile(inputpath)
#remove header
header = inputRDD.first()
inputRDD = inputRDD.filter(lambda x: x != header)

In [5]:
reviewsRDD = inputRDD.map(lambda x: x.split(",")).map(lambda x: (x[2], x[1])).distinct()

In [6]:
reviewsRDD.collect()

[('A2', 'B1'),
 ('A4', 'B1'),
 ('A2', 'B3'),
 ('A3', 'B3'),
 ('A4', 'B3'),
 ('A4', 'B4'),
 ('A5', 'B5'),
 ('A5', 'B1'),
 ('A1', 'B2'),
 ('A5', 'B3'),
 ('A2', 'B5'),
 ('A4', 'B5')]

In [7]:
productviewsRDD = reviewsRDD.groupByKey().mapValues(lambda x: list(x))

In [8]:
productviewsRDD.collect()

[('A2', ['B1', 'B3', 'B5']),
 ('A4', ['B1', 'B3', 'B4', 'B5']),
 ('A3', ['B3']),
 ('A1', ['B2']),
 ('A5', ['B5', 'B1', 'B3'])]

In [9]:
productviewsRDD.saveAsTextFile(outputpath1)

In [37]:
#Counts the frequencies of all the pairs of products reviewed together and merges the values of the same key (a,b) == (b,a)
def countPairs(x):
    pairs = []
    for i in range(len(x)):
        for j in range(i+1, len(x)):
            pairs.append((x[i], x[j]))
                
    return pairs
countPairsRDD = productviewsRDD.flatMapValues(lambda x: (countPairs(x))).map(lambda x: (x[1], 1))
countPairsRDD.collect()



[(('B1', 'B3'), 1),
 (('B1', 'B5'), 1),
 (('B3', 'B5'), 1),
 (('B1', 'B3'), 1),
 (('B1', 'B4'), 1),
 (('B1', 'B5'), 1),
 (('B3', 'B4'), 1),
 (('B3', 'B5'), 1),
 (('B4', 'B5'), 1),
 (('B5', 'B1'), 1),
 (('B5', 'B3'), 1),
 (('B1', 'B3'), 1)]

In [ ]:
# countPairsRDD = countPairsRDD.map(lambda x: (x[0][1], (x[0][0], x[1]))).union(countPairsRDD).reduceByKey(lambda x, y: x + y).map(lambda x: (x[0], x[1][0], x[1][1]))

In [40]:
countPairsRDD = countPairsRDD.map(lambda x: (x[0][1], (x[0][0], x[1]))).union(countPairsRDD.map(lambda x: (x[0][0], (x[0][1], x[1])))).reduceByKey(lambda x, y: x + y).map(lambda x: (x[0], x[1][0], x[1][1]))

In [41]:
countPairsRDD.collect()

[('4', 'B', ('B1', 1)),
 ('3', 'B', ('B1', 1)),
 ('B', '3', ('B1', 1)),
 ('1', 'B', ('B5', 1)),
 ('5', 'B', ('B1', 1))]